In [17]:
from typing import List, Dict, Set

class DocumentService:
    def __init__(self):
        self.documents = {}
        self.index = {}

    def add_document(self, title: str, content: str) -> str:
        doc_id = str(len(self.documents) + 1)
        self.documents[doc_id] = {
            'title': title,
            'content': content,
            'score': 0
        }
        self._index_document(doc_id, content)
        return doc_id

    def _index_document(self, doc_id: str, content: str) -> None:
        words = content.lower().split()
        for word in words:
            if word not in self.index:
                self.index[word] = set()
            self.index[word].add(doc_id)

    def get_document(self, doc_id: str) -> Dict[str, str]:
        """Retrieve a document by ID"""
        return self.documents.get(doc_id, {})

    def search_word(self, word: str) -> List[str]:
        """Find documents containing a word"""
        word = word.lower()
        return list(self.index.get(word, set()))

    def search_documents(self, query: str) -> List[Dict[str, str]]:
        # Split the query into individual conditions (AND, OR)
        terms = query.lower().split()
        doc_scores = {}

        # Separate AND and OR sections
        current_and_set = None
        current_or_set = set()

        for term in terms:
            if term == 'and':
                continue
            elif term == 'or':
                # Add the current AND group to the OR set
                if current_and_set is not None:
                    current_or_set.update(current_and_set)
                current_and_set = None
            else:
                # Process individual terms
                docs = set(self.search_word(term))
                if current_and_set is None:
                    current_and_set = docs
                else:
                    current_and_set &= docs

        # Add final AND set to the OR set (if any)
        if current_and_set is not None:
            current_or_set.update(current_and_set)

        # Calculate scores based on matching documents
        for doc_id in current_or_set:
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1

        # Sort by score (higher score = more relevance)
        ranked_results = sorted(
            [{'id': doc_id, 'title': self.get_document(doc_id)['title'], 'score': score} for doc_id, score in doc_scores.items()],
            key=lambda x: x['score'],
            reverse=True
        )
        return ranked_results


def main():
    service = DocumentService()
    service.add_document("Python Basics", "Python is a popular programming language.")
    service.add_document("Cloud Computing", "Cloud computing is the future of scalable applications.")
    service.add_document("Microservices Design", "Microservices architecture is a way to build scalable systems.")

    print("Search Results (AND/OR):")
    results = service.search_documents("Python and cloud or microservices")
    for result in results:
        print(f"{result['title']} (Score: {result['score']})")

    print("\nSearch Results with OR only:")
    results = service.search_documents("Python or cloud")
    for result in results:
        print(f"{result['title']} (Score: {result['score']})")


if __name__ == '__main__':
    main()


Search Results (AND/OR):
Microservices Design (Score: 1)

Search Results with OR only:
Python Basics (Score: 1)
Cloud Computing (Score: 1)
